In [1]:
import pandas as pd
import gmaps
import requests

from config import gkey

In [2]:
#store cities csv into dataframe

city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.dtypes


City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gkey)

In [4]:
#get the maximum temperature
max_temps = city_data_df['Max Temp']
temps = []
for temp in max_temps:
    temps.append(max(temp,0))

In [5]:
#heatmap of temperature
locations = city_data_df[['Lat','Lng']]
#get max temperature
max_temp = city_data_df['Max Temp']
#assign the figure variable
fig = gmaps.figure(center = (30.0,31.0), zoom_level=1.4)
#assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius = 4)
#add the heatmap layer
fig.add_layer(heat_layer)
#call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#heatmap of percent humidity
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.4)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#heatmap of percent cloudiness
locations = city_data_df[['Lat','Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.4)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#heatmap of  wind speed
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.4)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#ask the customer to add a min and max temp value
min_temp = float(input('What is the minimum temperature you would like for you trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))


What is the minimum temperature you would like for you trip?76
What is the maximum temperature you would like for your trip?88


In [10]:
#filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp)&\
                                      (city_data_df['Max Temp']>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
10,10,Puerto Ayora,EC,2022-04-23 18:05:46,-0.7393,-90.3518,76.96,79,34,9.53
12,12,Bambous Virieux,MU,2022-04-23 18:05:47,-20.3428,57.7575,77.25,69,75,17.27
14,14,Samarai,PG,2022-04-23 18:05:48,-10.6167,150.6667,81.73,79,58,9.98
19,19,Faanui,PF,2022-04-23 18:05:49,-16.4833,-151.7500,80.44,74,99,6.64
23,23,Leisure City,US,2022-04-23 18:05:51,25.4954,-80.4292,84.22,63,20,20.71
25,25,Victoria,HK,2022-04-23 18:05:47,22.2855,114.1577,78.96,89,9,4.00
27,27,Naliya,IN,2022-04-23 18:05:52,23.2667,68.8333,80.80,73,50,9.40
28,28,Harper,LR,2022-04-23 18:05:53,4.3750,-7.7169,82.45,76,98,7.20
32,32,Butaritari,KI,2022-04-23 18:05:55,3.0707,172.7902,81.64,75,53,15.70
38,38,Arraial Do Cabo,BR,2022-04-23 18:05:57,-22.9661,-42.0278,87.66,55,0,14.97


In [11]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [12]:
#make copy of preferred cities dataframe
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
12,Bambous Virieux,MU,77.25,-20.3428,57.7575,
14,Samarai,PG,81.73,-10.6167,150.6667,
19,Faanui,PF,80.44,-16.4833,-151.7500,
23,Leisure City,US,84.22,25.4954,-80.4292,
25,Victoria,HK,78.96,22.2855,114.1577,
27,Naliya,IN,80.80,23.2667,68.8333,
28,Harper,LR,82.45,4.3750,-7.7169,
32,Butaritari,KI,81.64,3.0707,172.7902,
38,Arraial Do Cabo,BR,87.66,-22.9661,-42.0278,


In [13]:
#set parameters to search for a hotel
params = {'radius':5000,
          'type':'lodging',
          'key':gkey
}
for index, row in hotel_df.iterrows():
    #get the lat and lng
    lat = row['Lat']
    lng = row['Lng']
    
    #add the lat and lng to location key for the params dictionary
    params['location']=f'{lat},{lng}'
    
    #use the search term : 'lodging' and our lat and lng
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #make requests and get the JSON data
    hotels = requests.get(base_url, params=params).json()
    #grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping.')




Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [14]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Bambous Virieux,MU,77.25,-20.3428,57.7575,Casa Tia Villa
14,Samarai,PG,81.73,-10.6167,150.6667,Nuli Sapi
19,Faanui,PF,80.44,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
23,Leisure City,US,84.22,25.4954,-80.4292,Hampton Inn & Suites Miami-South/Homestead


In [17]:
#add a heatmap of termparature for the vacation spots.
locations= hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.4)
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]